## Simple Binary Classification with defaults

In this notebook we will use the Adult Census dataset. Download the data from [here](https://www.kaggle.com/wenruliu/adult-income-dataset/downloads/adult.csv/2).

In [290]:
import numpy as np
import pandas as pd
import torch

from pytorch_widedeep.preprocessing import WidePreprocessor, TabPreprocessor
from pytorch_widedeep.training import Trainer
from pytorch_widedeep.models import Wide, TabMlp, TabResnet, TabTransformer, WideDeep
from pytorch_widedeep.metrics import Accuracy, Precision

from pytorch_widedeep.models.tab_transformer import IntersampleAttention, MultiHeadedAttention

d:\projects\Chris Projects\PODIUM\.venv\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [291]:
use_cuda = torch.cuda.is_available()
print(use_cuda)
device = torch.device("cuda" if use_cuda else "cpu")

False


In [276]:
"""
The code in this module is inspired by a number of implementations:

Classes PositionwiseFF and AddNorm are 'stolen' with much gratitude from the fantastic d2l.ai book:
https://d2l.ai/chapter_attention-mechanisms/transformer.html

MultiHeadedAttention is inspired by the TabTransformer implementation here:
https://github.com/lucidrains/tab-transformer-pytorch. General comment: just go and have a look to
https://github.com/lucidrains

The fixed attention implementation and SharedEmbeddings are inspired by the
TabTransformer available in AutoGluon:
https://github.com/awslabs/autogluon/tree/master/tabular/src/autogluon/tabular/models/tab_transformer
If you have not checked that library, you should.
"""

import math

import torch
import einops
from torch import nn, einsum

from pytorch_widedeep.wdtypes import *  # noqa: F403
from pytorch_widedeep.models.tab_mlp import MLP, _get_activation_fn

class ContinuousEmbedding(nn.Module):
  def __init__(
      self,
      embedding_size: int
  ):
      super(ContinuousEmbedding, self).__init__()
      self.relu = nn.ReLU()
      self.embedding = nn.Linear(1, embedding_size)

  def forward(self, X: Tensor) -> Tensor:
      return self.relu(self.embedding(X.t().unsqueeze(2)).permute(1, 0, 2))


class PositionwiseFF(nn.Module):
    def __init__(
        self,
        input_dim: int,
        ff_hidden_dim: int,
        dropout: float,
        activation: str,
    ):
        super(PositionwiseFF, self).__init__()
        self.w_1 = nn.Linear(input_dim, ff_hidden_dim)
        self.w_2 = nn.Linear(ff_hidden_dim, input_dim)
        self.dropout = nn.Dropout(dropout)
        self.activation = _get_activation_fn(activation)

    def forward(self, X: Tensor) -> Tensor:
        return self.w_2(self.dropout(self.activation(self.w_1(X))))


class AddNorm(nn.Module):
    def __init__(self, input_dim: int, dropout: float):
        super(AddNorm, self).__init__()
        self.dropout = nn.Dropout(dropout)
        self.ln = nn.LayerNorm(input_dim)

    def forward(self, X: Tensor, Y: Tensor) -> Tensor:
        return self.ln(self.dropout(Y) + X)


class MultiHeadedAttention(nn.Module):
    def __init__(
        self,
        input_dim: int,
        num_heads: int,
        keep_attn_weights: bool,
        dropout: float,
        fixed_attention: bool,
        num_cat_columns: int,
    ):
        super(MultiHeadedAttention, self).__init__()

        assert (
            input_dim % num_heads == 0
        ), "'input_dim' must be divisible by 'num_heads'"
        if fixed_attention and not num_cat_columns:
            raise ValueError(
                "if 'fixed_attention' is 'True' the number of categorical "
                "columns 'num_cat_columns' must be specified"
            )
        # Consistent with other implementations I assume d_v = d_k
        self.d_k = input_dim // num_heads
        self.num_heads = num_heads
        self.dropout = nn.Dropout(dropout)
        self.fixed_attention = fixed_attention
        if fixed_attention:
            self.inp_proj = nn.Linear(input_dim, input_dim)
            self.fixed_key = nn.init.xavier_normal_(
                nn.Parameter(torch.empty(num_cat_columns, input_dim))
            )
            self.fixed_query = nn.init.xavier_normal_(
                nn.Parameter(torch.empty(num_cat_columns, input_dim))
            )
        else:
            self.inp_proj = nn.Linear(input_dim, input_dim * 3)
        self.out_proj = nn.Linear(input_dim, input_dim)
        self.keep_attn_weights = keep_attn_weights

    def forward(self, X: Tensor) -> Tensor:
        # b: batch size, s: src seq length (num of categorical features
        # encoded as embeddings), l: target sequence (l = s), e: embeddings
        # dimensions, h: number of attention heads, d: d_k
        if self.fixed_attention:
            v = self.inp_proj(X)
            k = einops.repeat(
                self.fixed_key.unsqueeze(0), "b s e -> (b copy) s e", copy=X.shape[0]
            )
            q = einops.repeat(
                self.fixed_query.unsqueeze(0), "b s e -> (b copy) s e", copy=X.shape[0]
            )
        else:
            q, k, v = self.inp_proj(X).chunk(3, dim=2)
        q, k, v = map(
            lambda t: einops.rearrange(t, "b s (h d) -> b h s d", h=self.num_heads),
            (q, k, v),
        )
        scores = einsum("b h s d, b h l d -> b h s l", q, k) / math.sqrt(self.d_k)
        attn_weights = self.dropout(scores.softmax(dim=-1))
        if self.keep_attn_weights:
            self.attn_weights = attn_weights
        attn_output = einsum("b h s l, b h l d -> b h s d", attn_weights, v)
        output = einops.rearrange(attn_output, "b h s d -> b s (h d)", h=self.num_heads)

        return self.out_proj(output)

class IntersampleAttention(nn.Module):
    def __init__(
        self,
        input_dim: int,
        num_heads: int,
        keep_attn_weights: bool,
        dropout: float,
        fixed_attention: bool,
        num_cat_columns: int,
        num_input_columns: int
    ):
        super(IntersampleAttention, self).__init__()

        assert (
            input_dim % num_heads == 0
        ), "'input_dim' must be divisible by 'num_heads'"
        if fixed_attention and not num_cat_columns:
            raise ValueError(
                "if 'fixed_attention' is 'True' the number of categorical "
                "columns 'num_cat_columns' must be specified"
            )
        # Consistent with other implementations I assume d_v = d_k
        self.input_dim = input_dim
        self.d_k = input_dim // num_heads
        self.num_heads = num_heads
        self.dropout = nn.Dropout(dropout)
        self.fixed_attention = fixed_attention
        if fixed_attention:
            self.inp_proj = nn.Linear(input_dim, input_dim)
            self.fixed_key = nn.init.xavier_normal_(
                nn.Parameter(torch.empty(num_cat_columns, input_dim))
            )
            self.fixed_query = nn.init.xavier_normal_(
                nn.Parameter(torch.empty(num_cat_columns, input_dim))
            )
        else:
            self.inp_proj = nn.Linear(input_dim * num_input_columns, input_dim * num_input_columns * 3)
        self.out_proj = nn.Linear(input_dim, input_dim)
        self.keep_attn_weights = keep_attn_weights

    def forward(self, X):
        # b: batch size, s: src seq length (num of categorical features
        # encoded as embeddings), l: target sequence (l = s), e: embeddings
        # dimensions, h: number of attention heads, d: d_k
        # df: embedding dim * num of features
        X = einops.rearrange(X, "b s e -> () b (s e)")
        if self.fixed_attention:
            v = self.inp_proj(X)
            k = einops.repeat(
                self.fixed_key.unsqueeze(0), "b s e -> (b copy) s e", copy=X.shape[0]
            )
            q = einops.repeat(
                self.fixed_query.unsqueeze(0), "b s e -> (b copy) s e", copy=X.shape[0]
            )
        else:
            q, k, v = self.inp_proj(X).chunk(3, dim=2)
            
        q, k, v = map(
            lambda t: einops.rearrange(t, "() b (h df) -> () h b df", h=self.num_heads),
            (q, k, v),
        )
        scores = einsum("b h s d, b h l d -> b h s l", q, k) / math.sqrt(self.d_k)
        attn_weights = self.dropout(scores.softmax(dim=-1))
        if self.keep_attn_weights:
            self.attn_weights = attn_weights
        attn_output = einsum("b h s l, b h l d -> b h s d", attn_weights, v)
        output = einops.rearrange(attn_output, "() h b df -> () b (h df)", h=self.num_heads)
        output = einops.rearrange(output, "() b (s e) -> b s e", e = self.input_dim)

        return self.out_proj(output)
        
class SAINTTransformerEncoder(nn.Module):
    def __init__(
        self,
        input_dim: int,
        num_heads: int,
        keep_attn_weights: bool,
        ff_hidden_dim: int,
        dropout: float,
        activation: str,
        fixed_attention,
        num_cat_columns,
        num_input_columns,
    ):
        super(TransformerEncoder, self).__init__()
        self.self_attn = MultiHeadedAttention(
            input_dim,
            num_heads,
            keep_attn_weights,
            dropout,
            fixed_attention,
            num_cat_columns,
        )
        self.is_attn = IntersampleAttention(
            input_dim,
            num_heads,
            keep_attn_weights,
            dropout,
            fixed_attention,
            num_cat_columns,
            num_input_columns,
        )
        self.feed_forward = PositionwiseFF(
            input_dim, ff_hidden_dim, dropout, activation
        )
        self.attn_addnorm = AddNorm(input_dim, dropout)
        self.is_attn_addnorm = AddNorm(input_dim, dropout)
        self.ff_addnorm1 = AddNorm(input_dim, dropout)
        self.ff_addnorm2 = AddNorm(input_dim, dropout)

    def forward(self, X: Tensor) -> Tensor:
        Y_self_attn = self.self_attn(X)
        Y = self.attn_addnorm(X, Y_self_attn)
        Y = self.ff_addnorm1(Y, self.feed_forward(Y))
        Y_is_attn = self.is_attn(Y)
        Y = self.is_attn_addnorm(Y, Y_is_attn)
        return self.ff_addnorm2(Y, self.feed_forward(Y))


class FullEmbeddingDropout(nn.Module):
    def __init__(self, dropout: float):
        super(FullEmbeddingDropout, self).__init__()
        self.dropout = dropout

    def forward(self, X: Tensor) -> Tensor:
        mask = X.new().resize_((X.size(1), 1)).bernoulli_(1 - self.dropout).expand_as(
            X
        ) / (1 - self.dropout)
        return mask * X


class SharedEmbeddings(nn.Module):
    def __init__(
        self,
        num_embed: int,
        embed_dim: int,
        embed_dropout: float,
        full_embed_dropout: bool = False,
        add_shared_embed: bool = False,
        frac_shared_embed=8,
    ):
        super(SharedEmbeddings, self).__init__()
        assert (
            embed_dim % frac_shared_embed == 0
        ), "'embed_dim' must be divisible by 'frac_shared_embed'"

        self.add_shared_embed = add_shared_embed
        self.embed = nn.Embedding(num_embed, embed_dim, padding_idx=0)
        self.embed.weight.data.clamp_(-2, 2)
        if add_shared_embed:
            col_embed_dim = embed_dim
        else:
            col_embed_dim = embed_dim // frac_shared_embed
        self.shared_embed = nn.Parameter(torch.empty(1, col_embed_dim).uniform_(-1, 1))

        if full_embed_dropout:
            self.dropout = FullEmbeddingDropout(embed_dropout)
        else:
            self.dropout = nn.Dropout(embed_dropout)  # type: ignore[assignment]

    def forward(self, X: Tensor) -> Tensor:
        out = self.dropout(self.embed(X))
        shared_embed = self.shared_embed.expand(out.shape[0], -1)
        if self.add_shared_embed:
            out += shared_embed
        else:
            out[:, : shared_embed.shape[1]] = shared_embed
        return out


class SAINT(nn.Module):
    def __init__(
        self,
        column_idx: Dict[str, int],
        embed_input: List[Tuple[str, int]],
        continuous_cols: Optional[List[str]] = None,
        embed_dropout: float = 0.1,
        full_embed_dropout: bool = False,
        shared_embed: bool = False,
        add_shared_embed: bool = False,
        frac_shared_embed: int = 8,
        input_dim: int = 32,
        num_heads: int = 8,
        num_blocks: int = 6,
        dropout: float = 0.1,
        keep_attn_weights: bool = False,
        fixed_attention: bool = False,
        num_cat_columns: Optional[int] = None,
        ff_hidden_dim: int = 32 * 4,
        transformer_activation: str = "gelu",
        mlp_hidden_dims: Optional[List[int]] = None,
        mlp_activation: str = "relu",
        mlp_batchnorm: bool = False,
        mlp_batchnorm_last: bool = False,
        mlp_linear_first: bool = True,
        embed_continuous: bool = True,
    ):

        r"""TabTransformer model (https://arxiv.org/pdf/2012.06678.pdf) model that
        can be used as the deeptabular component of a Wide & Deep model.


        Parameters
        ----------
        column_idx: Dict
            Dict containing the index of the columns that will be passed through
            the DeepDense model. Required to slice the tensors. e.g. {'education':
            0, 'relationship': 1, 'workclass': 2, ...}
        embed_input: List
            List of Tuples with the column name and number of unique values
            e.g. [(education, 11, 32), ...]
        continuous_cols: List, Optional, default = None
            List with the name of the numeric (aka continuous) columns
        embed_dropout: float, default = 0.1
            Dropout to be applied to the embeddings matrix
        full_embed_dropout: bool, default = False
            Boolean indicating if an entire embedding (i.e. the representation
            for one categorical column) will be dropped in the batch. See:
            ``pytorch_widedeep.model.tab_transformer.FullEmbeddingDropout``.
            If ``full_embed_dropout = True``, ``embed_dropout`` is ignored.
        shared_embed: bool, default = False
            The idea behind ``shared_embed`` is described in the Appendix A in the paper:
            `'The goal of having column embedding is to enable the model to distinguish the
            classes in one column from those in the other columns'`. In other words, the idea
            is to let the model learn which column is embedding at the time.
        add_shared_embed: bool, default = False,
            The two embedding sharing strategies are: 1) add the shared embeddings to the column
            embeddings or 2) to replace the first ``frac_shared_embed`` with the shared
            embeddings. See ``pytorch_widedeep.models.tab_transformer.SharedEmbeddings``
        frac_shared_embed: int, default = 8
            The fraction of embeddings that will be shared by all the different categories for
            one particular column.
        input_dim: int, default = 32
            The so-called *dimension of the model*. Is the number of embeddings used to encode
            the categorical columns
        num_heads: int, default = 8
            Number of attention heads per Transformer block
        num_blocks: int, default = 6
            Number of Transformer blocks
        dropout: float, default = 0.1
            Dropout that will be applied internally to the
            ``TransformerEncoder`` (see
            ``pytorch_widedeep.model.tab_transformer.TransformerEncoder``) and the
            output MLP
        keep_attn_weights: bool, default = False
            If set to ``True`` the model will store the attention weights in the ``attention_weights``
            attribute.
        fixed_attention: bool, default = False
            If set to ``True`` all the observations in a batch will have the
            same Key and Query. This implementation is inspired by the one
            available at the `Autogluon tabular library
            <https://github.com/awslabs/autogluon/blob/master/tabular/src/autogluon/tabular/models/tab_transformer/modified_transformer.py>`_.
        num_cat_columns: int, Optional, default = None
            If `fixed_attention` is set to ``True`` the number of categorical
            columns that will be encoded as embeddings must be specified
        ff_hidden_dim: int, default = 128
            Hidden dimension of the ``FeedForward`` Layer. See
            ``pytorch_widedeep.model.tab_transformer.FeedForward``.
        transformer_activation: str, default = "gelu"
            Transformer Encoder activation function
        mlp_hidden_dims: List, Optional, default = None
            MLP hidden dimensions. If not provided it will default to ``[4*l,
            2*l]`` where ``l`` is the mlp input dimension
        mlp_activation: str, default = "gelu"
            MLP activation function
        mlp_batchnorm: bool, default = False
            Boolean indicating whether or not to apply batch normalization to the
            dense layers
        mlp_batchnorm_last: bool, default = False
            Boolean indicating whether or not to apply batch normalization to the
            last of the dense layers
        mlp_linear_first: bool, default = False
            Boolean indicating whether the order of the operations in the dense
            layer. If ``True: [LIN -> ACT -> BN -> DP]``. If ``False: [BN -> DP ->
            LIN -> ACT]``
        embed_continuous: bool, default = False
            Boolean indicating whether or not continuous variables should also be
            embedded. If true, they will pass through a variable-specific linear
            layer with ReLU activation (see https://arxiv.org/abs/2106.01342)

        Attributes
        ----------
        embed_layers: ``nn.ModuleDict``
            Dict with the embeddings per column
        tab_transformer_blks: ``nn.Sequential``
            Sequence of Transformer blocks
        attention_weights: List
            List with the attention weights per block
        mlp: ``nn.Module``
            MLP component in the TabTransformer model
        output_dim: int
            The output dimension of the model. This is a required attribute
            neccesary to build the WideDeep class
        continuous_embedding ``nn.Module``
            Embeddings for the continuous variables

        Example
        --------
        >>> import torch
        >>> from pytorch_widedeep.models import TabTransformer
        >>> X_tab = torch.cat((torch.empty(5, 4).random_(4), torch.rand(5, 1)), axis=1)
        >>> colnames = ['a', 'b', 'c', 'd', 'e']
        >>> embed_input = [(u,i) for u,i in zip(colnames[:4], [4]*4)]
        >>> continuous_cols = ['e']
        >>> column_idx = {k:v for v,k in enumerate(colnames)}
        >>> model = TabTransformer(column_idx=column_idx, embed_input=embed_input, continuous_cols=continuous_cols)
        >>> out = model(X_tab)
        """
        super(TabTransformer, self).__init__()

        self.column_idx = column_idx
        self.embed_input = embed_input
        self.continuous_cols = continuous_cols
        self.embed_dropout = embed_dropout
        self.full_embed_dropout = full_embed_dropout
        self.shared_embed = shared_embed
        self.add_shared_embed = add_shared_embed
        self.frac_shared_embed = frac_shared_embed
        self.input_dim = input_dim
        self.num_heads = num_heads
        self.num_blocks = num_blocks
        self.dropout = dropout
        self.keep_attn_weights = keep_attn_weights
        self.fixed_attention = fixed_attention
        self.num_cat_columns = num_cat_columns
        self.ff_hidden_dim = ff_hidden_dim
        self.transformer_activation = transformer_activation
        self.mlp_hidden_dims = mlp_hidden_dims
        self.mlp_activation = mlp_activation
        self.mlp_batchnorm = mlp_batchnorm
        self.mlp_batchnorm_last = mlp_batchnorm_last
        self.mlp_linear_first = mlp_linear_first
        self.embed_continuous = embed_continuous

        # Embeddings: val + 1 because 0 is reserved for padding/unseen cateogories.
        if shared_embed:
            self.embed_layers = nn.ModuleDict(
                {
                    "emb_layer_"
                    + col: SharedEmbeddings(
                        val + 1,
                        input_dim,
                        embed_dropout,
                        full_embed_dropout,
                        add_shared_embed,
                        frac_shared_embed,
                    )
                    for col, val in self.embed_input
                }
            )
        else:
            self.embed_layers = nn.ModuleDict(
                {
                    "emb_layer_" + col: nn.Embedding(val + 1, input_dim, padding_idx=0)
                    for col, val in self.embed_input
                }
            )
            if full_embed_dropout:
                self.embedding_dropout = FullEmbeddingDropout(embed_dropout)
            else:
                self.embedding_dropout = nn.Dropout(embed_dropout)  # type: ignore[assignment]
        # Continuous
        if self.continuous_cols is not None:
            cont_inp_dim = len(self.continuous_cols)
        else:
            cont_inp_dim = 0

        if self.embed_continuous:
            self.continuous_embedding = ContinuousEmbedding(self.input_dim)

        if embed_continuous:
            num_input_columns = len(embed_input) + cont_inp_dim
        else:
            num_input_columns = len(embed_input)

        self.tab_transformer_blks = nn.Sequential()
        for i in range(num_blocks):
            self.tab_transformer_blks.add_module(
                "block" + str(i),
                SAINTTransformerEncoder(
                    input_dim,
                    num_heads,
                    keep_attn_weights,
                    ff_hidden_dim,
                    dropout,
                    transformer_activation,
                    fixed_attention,
                    num_cat_columns,
                    num_input_columns,
                ),
            )

        if keep_attn_weights:
            self.attention_weights = [None] * num_blocks

        if not mlp_hidden_dims:
            if embed_continuous:
                mlp_inp_l = (len(embed_input) + cont_inp_dim) * input_dim
            else:
                mlp_inp_l = (len(embed_input) * input_dim) + cont_inp_dim
            mlp_hidden_dims = [mlp_inp_l, mlp_inp_l * 4, mlp_inp_l * 2]

        self.tab_transformer_mlp = MLP(
            mlp_hidden_dims,
            mlp_activation,
            dropout,
            mlp_batchnorm,
            mlp_batchnorm_last,
            mlp_linear_first,
        )

        # the output_dim attribute will be used as input_dim when "merging" the models
        self.output_dim = mlp_hidden_dims[-1]

    def forward(self, X: Tensor) -> Tensor:

        embed = [
            self.embed_layers["emb_layer_" + col](
                X[:, self.column_idx[col]].long()
            ).unsqueeze(1)
            for col, _ in self.embed_input
        ]
        x = torch.cat(embed, 1)
        if not self.shared_embed and self.embedding_dropout is not None:
            x = self.embedding_dropout(x)

        if self.continuous_cols is not None and self.embed_continuous:
            cont_idx = [self.column_idx[col] for col in self.continuous_cols]
            x_cont = X[:, cont_idx].float()
            x_cont = self.continuous_embedding(x_cont)
            x_cont = self.embedding_dropout(x_cont)
            x = torch.cat([x, x_cont], 1)

        for i, blk in enumerate(self.tab_transformer_blks):
            x = blk(x)
            if self.keep_attn_weights:
                self.attention_weights[i] = blk.self_attn.attn_weights
        x = x.flatten(1)

        if self.continuous_cols is not None and self.embed_continuous==False:
            cont_idx = [self.column_idx[col] for col in self.continuous_cols]
            x_cont = X[:, cont_idx].float()
            x = torch.cat([x, x_cont], 1)

        return self.tab_transformer_mlp(x)


In [277]:
is_attn = IntersampleAttention(
        input_dim = 32,
        num_heads = 8,
        keep_attn_weights = False,
        dropout = 0.1,
        fixed_attention = False,
        num_cat_columns = 3,
        num_input_columns = 6)

torch_test = torch.rand([91, 6, 32])

out = is_attn(torch_test)
out.shape

torch.Size([91, 6, 32])

In [278]:
df = pd.read_csv('data/adult.csv')
df.head()

d:\projects\Chris Projects\PODIUM\.venv\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [279]:
# For convenience, we'll replace '-' with '_'
df.columns = [c.replace("-", "_") for c in df.columns]
# binary target
df['income_label'] = (df["income"].apply(lambda x: ">50K" in x)).astype(int)
df.drop('income', axis=1, inplace=True)
df.head()

d:\projects\Chris Projects\PODIUM\.venv\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,age,workclass,fnlwgt,education,educational_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_label
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,0
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,0
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,1
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,1
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,0


### Preparing the data

Have a look to notebooks one and two if you want to get a good understanding of the next few lines of code (although there is no need to use the package)

In [280]:
wide_cols = ['education', 'relationship','workclass','occupation','native_country','gender']
crossed_cols = [('education', 'occupation'), ('native_country', 'occupation')]
cat_embed_cols = [('education',16), ('relationship',8), ('workclass',16), ('occupation',16),('native_country',16)]
continuous_cols = ["age","hours_per_week"]
target_col = 'income_label'

d:\projects\Chris Projects\PODIUM\.venv\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [281]:
# TARGET
target = df[target_col].values

# wide
wide_preprocessor = WidePreprocessor(wide_cols=wide_cols, crossed_cols=crossed_cols)
X_wide = wide_preprocessor.fit_transform(df)

# deeptabular
tab_preprocessor = TabPreprocessor(embed_cols=cat_embed_cols, continuous_cols=continuous_cols)
X_tab = tab_preprocessor.fit_transform(df)

In [282]:
print(X_wide)
print(X_wide.shape)

[[  1  17  23 ...  89  91 316]
 [  2  18  23 ...  89  92 317]
 [  3  18  24 ...  89  93 318]
 ...
 [  2  20  23 ...  90 103 323]
 [  2  17  23 ...  89 103 323]
 [  2  21  29 ...  90 115 324]]
(48842, 8)
d:\projects\Chris Projects\PODIUM\.venv\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [283]:
print(X_tab)
print(X_tab.shape)

[[ 1.          1.          1.         ...  1.         -0.99512893
  -0.03408696]
 [ 2.          2.          1.         ...  1.         -0.04694151
   0.77292975]
 [ 3.          2.          2.         ...  1.         -0.77631645
  -0.03408696]
 ...
 [ 2.          4.          1.         ...  1.          1.41180837
  -0.03408696]
 [ 2.          1.          1.         ...  1.         -1.21394141
  -1.64812038]
 [ 2.          5.          7.         ...  1.          0.97418341
  -0.03408696]]
(48842, 7)


### Defining the model

Using `TabTransformer` as the `deeptabular` component

In [284]:
# for TabTransformer we only need the names of the columns
cat_embed_cols_for_transformer = [el[0] for el in cat_embed_cols]

In [285]:
cat_embed_cols_for_transformer

['education', 'relationship', 'workclass', 'occupation', 'native_country']

In [286]:
# deeptabular
tab_preprocessor = TabPreprocessor(embed_cols=cat_embed_cols_for_transformer, 
                                   continuous_cols=continuous_cols, 
                                   for_tabtransformer=True)
X_tab = tab_preprocessor.fit_transform(df)

In [287]:
wide = Wide(wide_dim=np.unique(X_wide).shape[0], pred_dim=1)
deeptabular = TabTransformer(column_idx=tab_preprocessor.column_idx,
                             embed_input=tab_preprocessor.embeddings_input,
                             continuous_cols=continuous_cols,
                             embed_continuous=True)
model = WideDeep(wide=wide, deeptabular=deeptabular)

d:\projects\Chris Projects\PODIUM\.venv\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [288]:
trainer = Trainer(model, objective='binary', metrics=[Accuracy, Precision])

In [289]:
trainer.fit(X_wide=X_wide, X_tab=X_tab, target=target, n_epochs=10, batch_size=128, val_split=0.2)

valid: 100%|██████████| 77/77 [00:53<00:00,  1.45it/s, loss=0.349, metrics={'acc': 0.839, 'prec': 0.6959}]


Also mentioning that one could build a model with the individual components independently. For example, a model comprised only by the `wide` component would be simply a linear model. This could be attained by just:

In [15]:
wide = Wide(wide_dim=np.unique(X_wide).shape[0], pred_dim=1)
deeptabular = TabTransformer(column_idx=tab_preprocessor.column_idx,
                             embed_input=tab_preprocessor.embeddings_input,
                             continuous_cols=continuous_cols,
                             embed_continuous = False)
model = WideDeep(wide=wide, deeptabular=deeptabular)

In [16]:
trainer = Trainer(model, objective='binary', metrics=[Accuracy, Precision])

In [17]:
trainer.fit(X_wide=X_wide, X_tab=X_tab, target=target, n_epochs=5, batch_size=64, val_split=0.2)

valid: 100%|██████████| 153/153 [00:01<00:00, 98.96it/s, loss=0.353, metrics={'acc': 0.8363, 'prec': 0.6742}]


In [36]:
model = WideDeep(wide=wide)

In [37]:
trainer = Trainer(model, objective='binary', metrics=[Accuracy, Precision])

In [15]:
trainer.fit(X_wide=X_wide, target=target, n_epochs=10, batch_size=64, val_split=0.2)

epoch 1:   0%|          | 0/611 [00:00<?, ?it/s]


KeyError: 'deeptabular'

The only requisite is that the model component must be passed to `WideDeep` before "fed" to the `Trainer`. This is because the `Trainer` is coded so that it trains a model that has a parent called `model` and then children that correspond to the model components: `wide`,  `deeptabular`, `deeptext` and `deepimage`. Also, `WideDeep` builds the last connection between the output of those components and the final, output neuron(s).